# APIs Modules - ein erste Blick auf "Requests"

Gezielt und automatisiert Daten im Web auslesen

Zuerst installieren wir pandas und requests. Das können wir von der Command line mit den Befehlen ```pip install requests``` und ```pip install pandas``` die beiden Module installieren. Oder direkt hier in Jupyter Notebook mit ```!pip install requests``` und ```!pip install pandas```.

### Unsere Imports

In [4]:
import requests
import pandas as pd
import calendar, datetime, time
import datetime as dt

Zur Dokumentation von requestst geht es [hier lang](http://docs.python-requests.org/en/master/).

### Notes

In einer ersten Übung arbeiten wir mit [der API des USGS](https://earthquake.usgs.gov/fdsnws/event/1/#methods) arbeiten. Für die meisten API muss man sich heutezutage eine eigene API-Key kreieren. Das werden wir zu einem späteren Zeitpunkt tun. Diese API braucht keine Keys. Deshalb ist es ein gutes Kennenlernbeispiel.

Falls Probleme mit dem Output auftauchen, hier ein Work around für die Meldung, dass der Output limitiert wurde. [Reconfigure line output](https://stackoverflow.com/questions/43288550/iopub-data-rate-exceeded-when-viewing-image-in-jupyter-notebook) and [opening stuff from the command line](https://stackoverflow.com/questions/16344709/how-to-open-a-file-from-the-command-line-with-a-specified-program)

### Dokumentation 

Bevor man mit deiner API arbeitet, lohnt es sich immer zuerst, die [Dokumentation](https://earthquake.usgs.gov/fdsnws/event/1/#parameters) anzuschauen. 

### Erste Abfragen

Kleinere Abfrage. Wieviele Erdbeben hat es in einem bestimmten Zeitraum gegeben?

In [5]:
url = 'https://earthquake.usgs.gov/fdsnws/event/1/count?starttime=2017-09-20&endtime=2017-09-21'

In [6]:
response = requests.get(url)

In [7]:
response

<Response [200]>

In [8]:
response.text

'328'

Wieviele hat es letzte Nacht von 22 bis 6 Uhr morgens gegeben? Dafür müssen wir in der Dokumentation Time nachfragen.

In [9]:
url = 'https://earthquake.usgs.gov/fdsnws/event/1/count?starttime=2017-09-20&endtime=2017-09-21

SyntaxError: EOL while scanning string literal (<ipython-input-9-bf5bc0961c93>, line 1)

Wegen Zeitverschiebung Zeit anpassen.

In [ ]:
url = 'https://earthquake.usgs.gov/fdsnws/event/1/count?starttime=2017-09-20T24:00:00&endtime=2017-09-21T08:00:00'

In [ ]:
response = requests.get(url)
response.text

### Grosse, komibinierte Abfrage

Details zu allen Erbeben in den über drei Tage, 17-9-2017 bis 21-9-2017.

In [ ]:
url1 = 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson'

In [ ]:
urlzeit = '&starttime=2017-09-17T24:00:00&endtime=2017-09-21T08:00:00'

In [ ]:
url + urlzeit

In [ ]:
response = requests.get(url1+urlzeit)
response.text

Alle Erbeben mit den ungefähren Koordinaten von Mexiko. [Hier könnte man die ungefähr nachschlagen.](http://www.latlong.net/)

In [ ]:
urlloc = '&minlatitude=13&maxlatitude=33&minlongitude=-120&maxlongitude=-85'

In [ ]:
url1+urlzeit+urlloc

In [ ]:
response = requests.get(url1+urlzeit+urlloc)
response.text

Dieser Wulst lässt sich aber ziemlich schwer lesen. Deshalb müssen wir das zuerst umformatieren und dann verstehen, wie das ganze strukturiert ist. 

In [ ]:
response.json()

In [ ]:
dct = response.json()

Um sicher zu gehen, prüfen wir nochmals den Datentyp.

In [ ]:
type(dct)

In [ ]:
len(dct)

Schauen wir uns alle keys und values an

In [ ]:
for key in dct:
    print(key)

Features interessiert uns.

In [ ]:
type(dct['features'])

In [ ]:
len(dct['features'])

Schauen wir uns das erste Element an

In [ ]:
for key in dct['features'][0]:
    print(key)

In [ ]:
dct['features'][0]['properties']


Lesen wir folgendes aus:
1. Type
2. Magnitude
4. Örtlichkeit
5. Time 

In [ ]:
earthquakes = []

for elem in dct['features']:
    
    t = elem['properties']['type']
    m = elem['properties']['mag']
    p = elem['properties']['place']
    tm = elem['properties']['time']
    
    mini_dict = {'Type': t,
                 'Mag': m,
                 'Place': p,
                 'Time': tm}
    
    earthquakes.append(mini_dict)

Dealing with Unix time

In [ ]:
earthquakes

In [ ]:
earthquakes = []

for elem in dct['features']:
    t = elem['properties']['type']
    m = elem['properties']['mag']
    p = elem['properties']['place']
    
    #Ints bearbeiten, die letzten drei Stellen abschneiden
    tm = str(elem['properties']['time'])[:-3]
        
    mini_dict = {'Type': t,
                 'Mag': m,
                 'Place': p,
                 'Time': tm}
    
    earthquakes.append(mini_dict)

In [ ]:
zahl = 12121323234234234324234

In [ ]:
temp = datetime.datetime.fromtimestamp(1386181800).strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
temp

In [ ]:
str(zahl)

In [ ]:
dt(int(str(zahl)[:-3]))

In [ ]:
dt(int(str(zahl)[:-3])) 

In [ ]:
earthquakes

### Erste Erfahrung mit Pandas

In [ ]:
pd.DataFrame(earthquakes)

In [ ]:
df = pd.DataFrame(earthquakes)

In [ ]:
df.sort_values(by='Mag', ascending=False)

Speichern wir das ab.

In [ ]:
df.to_csv('erdbeben.csv')